In [1]:
import tensorflow as tf
print(tf.__version__)

2.0.0


In [20]:
K.clear_session()

In [2]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  1


In [3]:
import tensorflow as tf
from tensorflow.keras import Input
from tensorflow.keras.layers import Convolution2D, Layer, UpSampling2D, Conv2DTranspose, Flatten, Reshape, Lambda, BatchNormalization,Dense, Activation
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K
from tensorflow.keras.losses import mse, binary_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import load_img


import os
import numpy as np
import math
from PIL import Image

In [23]:
!pip install Pillow

In [4]:
image_size = 128
inputShape = (128, 128, 3)
latentSize = 64

#encoder_type deepmind_enc
data_dir = '/home/dovi/home/dataset'
graph_dir = '/content/drive/My Drive/Colab Notebooks/VAE/graphs'
test_image_folder = '/content/drive/My Drive/Colab Notebooks/VAE/test_images/celeb'
encoder_type = 'deepmind_enc'
save_dir = '/content/drive/My Drive/Colab Notebooks/VAE/saved_models'
save_dir_img = '/content/drive/My Drive/Colab Notebooks/VAE/new_vae_images'
val_split = 0.2

train_batch_size = 32
val_batch_size = 32 
optimizer = 'ADAM' 
base_learning_rate = 0.01
num_epochs = 100 
scheduler_epoch = 5 
decay_factor = 0.01 
capacity = 25
multi_process=True

In [5]:
def post_process_output(generated_image):
    generated_image[generated_image > 0.5] = 0.5
    generated_image[generated_image < -0.5] = -0.5
    gen_image = np.uint8((generated_image + 0.5) * 255)
    return gen_image


def batch_gen(generator_object):
    while True:
        data = generator_object.next()
        yield [pre_process_input(data[0])], [pre_process_input(data[0])]
      
      
def pre_process_input(image_array):
    x = np.asarray(image_array)
    y = x - 0.5
    return y

In [6]:
def plot_results(models,
                 data,
                 batch_size=128,
                 model_name="vae_mnist"):
    """Plots labels and MNIST digits as function of 2-dim latent vector

    # Arguments
        models (tuple): encoder and decoder models
        data (tuple): test data and label
        batch_size (int): prediction batch size
        model_name (string): which model is using this function
    """

    encoder, decoder = models
    x_test, y_test = data
    os.makedirs(model_name, exist_ok=True)

    filename = os.path.join(model_name, "vae_mean.png")
    # display a 2D plot of the digit classes in the latent space
    z_mean, _, _ = encoder.predict(x_test,
                                   batch_size=batch_size)
    plt.figure(figsize=(12, 10))
    plt.scatter(z_mean[:, 0], z_mean[:, 1], c=y_test)
    plt.colorbar()
    plt.xlabel("z[0]")
    plt.ylabel("z[1]")
    plt.savefig(filename)
    plt.show()

    filename = os.path.join(model_name, "digits_over_latent.png")
    # display a 30x30 2D manifold of digits
    n = 30
    digit_size = 28
    figure = np.zeros((digit_size * n, digit_size * n))
    # linearly spaced coordinates corresponding to the 2D plot
    # of digit classes in the latent space
    grid_x = np.linspace(-4, 4, n)
    grid_y = np.linspace(-4, 4, n)[::-1]

    for i, yi in enumerate(grid_y):
        for j, xi in enumerate(grid_x):
            z_sample = np.array([[xi, yi]])
            x_decoded = decoder.predict(z_sample)
            digit = x_decoded[0].reshape(digit_size, digit_size)
            figure[i * digit_size: (i + 1) * digit_size,
                   j * digit_size: (j + 1) * digit_size] = digit

    plt.figure(figsize=(10, 10))
    start_range = digit_size // 2
    end_range = n * digit_size + start_range + 1
    pixel_range = np.arange(start_range, end_range, digit_size)
    sample_range_x = np.round(grid_x, 1)
    sample_range_y = np.round(grid_y, 1)
    plt.xticks(pixel_range, sample_range_x)
    plt.yticks(pixel_range, sample_range_y)
    plt.xlabel("z[0]")
    plt.ylabel("z[1]")
    plt.imshow(figure, cmap='Greys_r')
    plt.savefig(filename)
    plt.show()

In [7]:
def sampling(args):
    """Implements reparameterization trick by sampling
    from a gaussian with zero mean and std=1.
    Arguments:
        args (tensor): mean and log of variance of Q(z|X)
    Returns:
        sampled latent vector (tensor)
    """

    z_mean, z_log_var = args
    batch = K.shape(z_mean)[0]
    dim = K.int_shape(z_mean)[1]
    # by default, random_normal has mean=0 and std=1.0
    epsilon = K.random_normal(shape=(batch, dim))
    return z_mean + K.exp(0.5 * z_log_var) * epsilon

In [8]:
def conv_func(input_layer):
    x = Convolution2D(filters=32, kernel_size=4, strides=2, padding='same')(input_layer)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    return x

z_dim = latentSize
inputs = Input(shape=inputShape, name='encoder_input')
x = inputs

for _ in range(5):
    x = conv_func(x)

x = Flatten()(x)

x = Dense(units=256)(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = Dense(units=256)(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = Dense(units=z_dim * 2, activation='linear')(x)
# Beta-VAE implementation as follows

z_mean = Dense(units=z_dim, activation='linear', name='sample_mean')(x)
z_log_var = Dense(units=z_dim, activation='linear', name='sample_log_var')(x)
z = Lambda(sampling, output_shape=(z_dim,), name='z')([z_mean, z_log_var])

encoder = Model(inputs=[inputs], outputs=[z_mean, z_log_var, z], name='encoder')
encoder.summary()

InternalError: CUDA runtime implicit initialization on GPU:0 failed. Status: out of memory

In [9]:
def deconv_func(input_layer, pad_mode='same'):
    x = Conv2DTranspose(32, 4, strides=2, padding=pad_mode)(input_layer)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    return x


latent_inputs = Input(shape=(z_dim,), name='z_sampling')
x = Dense(units=256)(latent_inputs)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = Dense(units=256)(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = Dense(units=512, activation='relu')(x)
x = BatchNormalization()(x)
x = Reshape((-1, 1, 512), input_shape=(512,))(x)

for i in range(5):
    if i == 0:
        x = deconv_func(x, pad_mode='valid')
    else:
        x = deconv_func(x)
x = Conv2DTranspose(3, 4, strides=2, padding='same', activation='tanh')(x)
decoder = Model(inputs=latent_inputs, outputs=x, name='decoder')
decoder.summary()

InternalError: CUDA runtime implicit initialization on GPU:0 failed. Status: out of memory

In [10]:
# Loss
beta = 10
def vae_loss(y_true, y_pred):
    reconstruction_loss = mse(K.flatten(inputs), K.flatten(outputs))
    reconstruction_loss *= image_size * image_size
    kl_loss = 1 + z_log_var - K.square(z_mean) - K.exp(z_log_var)
    kl_loss = K.sum(kl_loss, axis=-1)
    kl_loss *= -0.5 * beta
    vae_loss = K.mean(reconstruction_loss + kl_loss)
    return vae_loss

In [11]:
# instantiate VAE model
outputs = decoder(inputs)

NameError: name 'decoder' is not defined

In [38]:
vae = Model(inputs=[inputs], outputs=[outputs], name='vae')
vae.summary()

Model: "vae"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
encoder_input (InputLayer)   [(None, 128, 128, 3)]     0         
_________________________________________________________________
encoder (Model)              [(None, 64), (None, 64),  316448    
_________________________________________________________________
decoder (Model)              (None, None, 128, 3)      548131    
Total params: 864,579
Trainable params: 860,867
Non-trainable params: 3,712
_________________________________________________________________


In [39]:
data_gen = ImageDataGenerator(rescale=1 / 255.,validation_split=val_split)

train_generator = data_gen.flow_from_directory(data_dir, target_size=(image_size, image_size), batch_size=train_batch_size, class_mode='categorical', subset='training')

validation_generator = data_gen.flow_from_directory(data_dir, target_size=(image_size, image_size), batch_size=val_batch_size, class_mode='categorical', subset='validation')

training_steps = math.ceil(train_generator.n / train_batch_size)
validation_steps = math.ceil(validation_generator.n / val_batch_size)

print('Steps per epoch in training: ', training_steps)
print('Steps per epoch in validation: ', validation_steps)

training_generator = batch_gen(generator_object=train_generator)

validation_generator = batch_gen(generator_object=validation_generator)

Found 191197 images belonging to 1 classes.
Found 47799 images belonging to 1 classes.
Steps per epoch in training:  5975
Steps per epoch in validation:  1494


In [40]:
def my_vae_loss(y_true, y_pred):
    xent_loss = image_size * image_size * mse(K.flatten(inputs), K.flatten(outputs))
    kl_loss = - 0.5 * beta * K.sum(1 + z_log_var - K.square(z_mean) - K.exp(z_log_var), axis=-1)
    vae_loss = K.mean(xent_loss + kl_loss)
    return vae_loss

In [41]:
vae.compile(optimizer='rmsprop', loss=my_vae_loss)

In [42]:
vae.summary()

Model: "vae"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
encoder_input (InputLayer)   [(None, 128, 128, 3)]     0         
_________________________________________________________________
encoder (Model)              [(None, 64), (None, 64),  316448    
_________________________________________________________________
decoder (Model)              (None, None, 128, 3)      548131    
Total params: 864,579
Trainable params: 860,867
Non-trainable params: 3,712
_________________________________________________________________


In [43]:
vae.fit_generator(training_generator, steps_per_epoch=training_steps, epochs=num_epochs,validation_data=validation_generator,validation_steps=validation_steps)

Epoch 1/100


ValueError: No gradients provided for any variable: ['conv2d/kernel:0', 'conv2d/bias:0', 'batch_normalization/gamma:0', 'batch_normalization/beta:0', 'conv2d_1/kernel:0', 'conv2d_1/bias:0', 'batch_normalization_1/gamma:0', 'batch_normalization_1/beta:0', 'conv2d_2/kernel:0', 'conv2d_2/bias:0', 'batch_normalization_2/gamma:0', 'batch_normalization_2/beta:0', 'conv2d_3/kernel:0', 'conv2d_3/bias:0', 'batch_normalization_3/gamma:0', 'batch_normalization_3/beta:0', 'conv2d_4/kernel:0', 'conv2d_4/bias:0', 'batch_normalization_4/gamma:0', 'batch_normalization_4/beta:0', 'dense/kernel:0', 'dense/bias:0', 'batch_normalization_5/gamma:0', 'batch_normalization_5/beta:0', 'dense_1/kernel:0', 'dense_1/bias:0', 'batch_normalization_6/gamma:0', 'batch_normalization_6/beta:0', 'dense_2/kernel:0', 'dense_2/bias:0', 'sample_mean/kernel:0', 'sample_mean/bias:0', 'sample_log_var/kernel:0', 'sample_log_var/bias:0', 'dense_3/kernel:0', 'dense_3/bias:0', 'batch_normalization_7/gamma:0', 'batch_normalization_

In [24]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

InternalError: cudaGetDevice() failed. Status: CUDA driver version is insufficient for CUDA runtime version

In [31]:
print(tf.keras.__version__)

2.2.4-tf


In [55]:
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
session = Session(config = config)
tf.compat.v1.disable_eager_execution()

AttributeError: module 'tensorflow' has no attribute 'ConfigProto'